# Plotly's Python API User Guide

## Section 3: Bubble Charts

Welcome to Plotly's Python API User Guide. 

> Links to the other sections are on the User Guide's [homepage](https://plot.ly/python/user-guide#Table-of-Contents:) <br>
The Github repository is available 
<a href="https://github.com/plotly/python-user-guide" target="_blank">here</a>

Section 3 is divided as follows:

* [3.1 A Plotly version of Hans Rosling's bubble chart](#3.1-A-Plotly-version-of-Hans-Rosling's-bubble-chart)
* [3.2 Small multiple bubble chart](#3.2-Small-multiple-bubble-chart)

Quickstart (make a bubble charts of 3 bubbles):

    >>> import plotly.plotly as py
    >>> from plotly.graph_objs import *
    >>> # auto sign-in with credentials or use py.sign_in()
    >>> trace1 = Scatter(
            x=[1, 2, 3],
            y=[5, 6, 7],
            mode='markers',
            marker=Marker(
                color=['blue', 'red', 'yellow'],
                size=[10, 20, 30]
            )
        )
    >>> data = Data([trace1])
    >>> py.plot(data)

<hr>

Check which version is installed on your machine and please upgrade if needed. 

In [2]:
# (*) Import plotly package
import plotly

# Check plolty version (if not latest, please upgrade)
plotly.__version__

'3.6.0'

See the User Guide's <a href="https://plot.ly/python/user-guide#Installation-guidelines" target="_blank">homepage</a> for more info on installation and upgrading. 

<hr>

In this section, we analyze <a href="http://www.gapminder.org/" target="_blank">GapMinder</a> data of life expectancy and gross domestic product (GDP) of all the world's countries using bubble charts, inspired by <a href="http://en.wikipedia.org/wiki/Hans_Rosling" target="_blank">Hans Rosling's</a> work. We also cover some of Plotly's marker object options,

* [Axis types](#Axis-types) and
* [Hover-mode options](#Hover-mode-options). 


We first import a few modules and sign in to Plotly using our credentials file:

In [3]:
# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py  

# (*) Useful Python/Plotly tools
import plotly.tools as tls   

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

import numpy as np  # (*) numpy for math functions and arrays

If you are not familiar with credentials files, refer to the User Guide's  <a href="https://plot.ly/python/user-guide#Step-6" target="_blank">homepage</a>.

The data is stored in a file called `gapminderDataFiveYear.txt`. For readers looking to run this notebook, the file is in this section's Github <a href="https://github.com/plotly/python-user-guide/blob/master/s3_bubble-charts" target="_blank">folder</a>. It was originally downloaded <a href="http://www.stat.ubc.ca/~jenny/notOcto/STAT545A/examples/gapminder/data/gapminderDataFiveYear.txt" target="_blank">here</a>, thanks to <a href="http://www.stat.ubc.ca/~jenny/" target="_blank">Jennifer Bryan</a>. 

To help us manipulate the date, we use the `pandas` module in this section. <br>
You can download the module using pip. Simply run:

    $ pip install pandas
    
For more info on this module, we recommand Hernán Rojas' <a href="https://bitbucket.org/hrojas/learn-pandas" target="_blank">Pandas tutorial</a>. 

So, first read the data and define a dataframe object:

In [4]:
# (*) Pandas for data manipulation
import pandas as pd 

# Read csv file and define dataframe object (df for dataframe)
df = pd.read_csv('c:\pydata\gapminderDataFiveYear.csv')

df.head()  # show dataframe header to stdout

country  year         pop continent  lifeExp   gdpPercap
0  Afghanistan  1952   8425333.0      Asia   28.801  779.445314
1  Afghanistan  1957   9240934.0      Asia   30.332  820.853030
2  Afghanistan  1962  10267083.0      Asia   31.997  853.100710
3  Afghanistan  1967  11537966.0      Asia   34.020  836.197138
4  Afghanistan  1972  13079460.0      Asia   36.088  739.981106

The data file gives us information about life expentancy, GDP per capita and population for several years, continents and conuntry. Would it be too much to ask to condense all this information into a single plot? Using Plotly bubble chart, the answer is no.

#### 3.1 A Plotly version of Hans Rosling's bubble chart

Bubble charts a great for way to show information about several variables on a single plot. Bubble charts in Plotly are generated using the `Scatter` graph object where two variables are plotted against each other following a particular color scheme and marker size distribution (i.e. the bubbles) thus relating up to four different variables. 
 
So, let's plot:

* Life expentancy versus GDP per capita for a specific year, for all countries in the data file, filling in the `'y'` and `'x'` keys in `Scatter`.
* Marker sizes incresing with population size (the bubbles!), with the `'size'` key in the `Marker` object linked to `'marker'` in `Scatter`.
* Marker colors correspond to different continents with the `'color'` key in the `Marker` object linked to `'marker'` in `Scatter`.

Finally, to make sure that each bubble is visible, we modify the opacity of each marker. This is done by linking the `'opacity'` key in `Marker` to a value between 0 and 1.

In [5]:
help(Marker)  # call help()

# notice 'color', 'size', 'sizeref', 'sizemode' and 'opacity'

Help on class Marker in module plotly.graph_objs._deprecations:

class Marker(builtins.dict)
 |  Marker(*args, **kwargs)
 |  
 |      plotly.graph_objs.Marker is deprecated.
 |  Please replace it with one of the following more specific types
 |    - plotly.graph_objs.scatter.Marker
 |    - plotly.graph_objs.histogram.selected.Marker
 |    - etc.
 |  
 |  Method resolution order:
 |      Marker
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |              plotly.graph_objs.Marker is deprecated.
 |      Please replace it with one of the following more specific types
 |        - plotly.graph_objs.scatter.Marker
 |        - plotly.graph_objs.histogram.selected.Marker
 |        - etc.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak reference

Next, select one year and slice the dataframe correspondingly:

In [6]:
# Choose a year, find other years with df['year'].unique()
the_year = 2007   

# Find indices corresponding to 'the_year'
i_year = (df['year'] == the_year)

# Grab all rows correponding to 'the_year'
df_year = df[i_year] 

Define a dictionary of colors, one for each continent:

In [7]:
colors = dict(
    Asia='#1f77b4', 
    Europe='#ff7f0e', 
    Africa='#2ca02c',
    Americas='#d62728',
    Oceania='#9467bd'
)

We choose to plot data associated with each continent as a different trace. To taking advantage of the regularity between each trace, we next define a trace-generating function.

Here, `'size'` in `Marker` will be linked to an array with the population count of each country in a given continent. To fine-tune the map between to the values linked to `'size'` to rendered pixel size, we use the `'sizeref'` and `'size'` in `Marker`. Consider,

In [8]:
# (!) Set 'size' values to be proportional to rendered area,
#     instead of diameter. This makes the range of bubble sizes smaller
sizemode = 'area'       

# (!) Set a reference for 'size' values (i.e. a population-to-pixel scaling).
#     Here the max bubble area will be on the order of 100 pixels
sizeref = df_year['pop'].max() / 1e2**2

# Define a trace-generating function (returns a Scatter object)
def make_trace(X, continent, sizes, color):  
    return Scatter(
        x=X['gdpPercap'],  # GDP on the x-xaxis
        y=X['lifeExp'],    # life Exp on th y-axis
        name=continent,    # label continent names on hover
        mode='markers',    # (!) point markers only on this plot
        marker= Marker(
            color=color,          # marker color
            size=sizes,           # (!) marker sizes (sizes is a list)
            sizeref=sizeref,      # link sizeref
            sizemode=sizemode,    # link sizemode
            opacity=0.6,          # (!) partly transparent markers
            line=Line(width=0.0)  # remove marker borders
        )
    )

Next, fill the data objects:

In [9]:
# Initialize data object 
data = Data()

# Group data frame by continent sub-dataframe (named X), 
#   make one trace object per continent and append to data object
for continent, X in df_year.groupby('continent'):
    
    sizes = X['pop']                            # get population array 
    color = colors[continent]                   # get bubble color
    
    data.append(
        make_trace(X, continent, sizes, color)  # append trace to data object
    )                             

c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.his

Onto layout:

In [10]:
# Set plot and axis titles
title = "Fig 3.1a: Hans Rosling's Bubble Chart for the year {}".format(the_year)
x_title = "Gross Domestic Product per Capita [in USD of the year 2000]"
y_title = "Life Expentancy [in years]"

# Define a dictionary of axis style options
axis_style = dict(     
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    plot_bgcolor='#EFECEA',  # set plot color to grey
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
    )
)

c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




Package the data and layout instances into `Figure`, send it to Plotly and get a plot:

In [11]:
# Make Figure object
fig = Figure(data=data, layout=layout)

# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='s3_life-gdp')

c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Not bad!

Notice how Plotly automatically converted the x-axis units to k's (i.e. thousands) of USD. No need to change the ticks' labels here (imagine having tick labels of 0, 10000, 20000, 30000, etc. instead).

##### Axis types

More generally, Plotly allow users to choose between different *exponent format* with the `'exponentformat'` key in `XAxis` and `YAxis`. The available values for `'exponentformat'` are:

* `'none'`, for no exponents,
* `'e'`, for lower-case exponent form, e.g. `1000` is displayed as `1e-3`,
* `'E'`, for upper-case exponent form, e.g. `1000` is displayed as `1E-3`,
* `'power'`, for powers of ten exponents, e.g. `1000` is displayed as $10^3$,
* `'SI'`, for Standard International prefixes, e.g. `10,000` is `10k` and 1 billion is `1G`,
* `'B'`, for alternative prefixes e.g. 1 billion is `1B`.

The key `'showexponent'`, also in `XAxis` and `YAxis`, determines which ticks labels are displayed in exponent form. The available values for `'showexponent'` are:

* `'none'`, for no exponents,
* `'all'`, for exponents at all tick labels,
* `'first'`, for an exponent only at the first tick label,
* `'last'`, for an exponent only at the last tick label.

In our next plot, we'll set all our exponents to `'power'`.

In addition, as the relationship between GDP per capita and life expentancy appears to be logarithmic, we replot our first bubble chart on log x-axis using the `type` key. 

In [12]:
help(XAxis) # call help()!

# notice the 'type' key!

Help on class XAxis in module plotly.graph_objs._deprecations:

class XAxis(builtins.dict)
 |  XAxis(*args, **kwargs)
 |  
 |      plotly.graph_objs.XAxis is deprecated.
 |  Please replace it with one of the following more specific types
 |    - plotly.graph_objs.layout.XAxis
 |    - plotly.graph_objs.layout.scene.XAxis
 |  
 |  Method resolution order:
 |      XAxis
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |              plotly.graph_objs.XAxis is deprecated.
 |      Please replace it with one of the following more specific types
 |        - plotly.graph_objs.layout.XAxis
 |        - plotly.graph_objs.layout.scene.XAxis
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------

So, changing the axis type is as simple as adding a key-value pair in `XAxis` (or `YAxis`). 

##### Hover-mode options

Moreover, let's utilize Plotly's intractability by adding some hover text (the `'text'` key in `Scatter`) and tweaking the plot's *hover mode*.

In [13]:
help(Layout)  # call help()! 

# notice 'hovermode'!

Help on class Layout in module plotly.graph_objs._layout:

class Layout(plotly.basedatatypes.BaseLayoutType)
 |  Layout(arg=None, angularaxis=None, annotations=None, annotationdefaults=None, autosize=None, bargap=None, bargroupgap=None, barmode=None, barnorm=None, boxgap=None, boxgroupgap=None, boxmode=None, calendar=None, clickmode=None, colorscale=None, colorway=None, datarevision=None, direction=None, dragmode=None, editrevision=None, extendpiecolors=None, font=None, geo=None, grid=None, height=None, hiddenlabels=None, hiddenlabelssrc=None, hidesources=None, hoverdistance=None, hoverlabel=None, hovermode=None, images=None, imagedefaults=None, legend=None, mapbox=None, margin=None, meta=None, metasrc=None, modebar=None, orientation=None, paper_bgcolor=None, piecolorway=None, plot_bgcolor=None, polar=None, radialaxis=None, scene=None, selectdirection=None, selectionrevision=None, separators=None, shapes=None, shapedefaults=None, showlegend=None, sliders=None, sliderdefaults=None, spik

By default, when hovering over a data point Plotly prints its y-value near the cursor and its x-value on the x-axis. The default behavior corresponds to `'hovermode':'x'` in `Layout`. In contrast, with `'hovermode':'y'`,  Plotly prints x-values near the cursor and y-values on the y-axis.

Alternatively, when `'hovermode'` is set to `'closest'`, information about the data point closest to where the viewer is hovering appears next to the cursor. Plotly prints this info inside the same text box as the text linked to the `'text'` key (if defined) of the trace object in question.

So, we make a few updates to the current figure object:

In [14]:
# Update 'xaxis' key, set it to log type and with a power exponent format
fig['layout']['xaxis'].update(
    type='log',
    exponentformat='power',
    showexponent='all'
)

# Update the layout object
fig['layout'].update(
    hovermode='closest',  # (!) hover -> closest data pt
    showlegend=False,     # remove legend (info in hover)
    autosize=False,       # turn off autosize
    width=650,            # plot width
    height=500,           # plot height
)

Layout({
    'autosize': False,
    'height': 500,
    'hovermode': 'closest',
    'plot_bgcolor': '#EFECEA',
    'showlegend': False,
    'title': {'text': "Fig 3.1a: Hans Rosling's Bubble Chart for the year 2007"},
    'width': 650,
    'xaxis': {'exponentformat': 'power',
              'gridcolor': '#FFFFFF',
              'showexponent': 'all',
              'ticklen': 8,
              'ticks': 'outside',
              'tickwidth': 1.5,
              'title': {'text': 'Gross Domestic Product per Capita [in USD of the year 2000]'},
              'type': 'log',
              'zeroline': False},
    'yaxis': {'gridcolor': '#FFFFFF',
              'ticklen': 8,
              'ticks': 'outside',
              'tickwidth': 1.5,
              'title': {'text': 'Life Expentancy [in years]'},
              'zeroline': False}
})

Next, add hover text presenting the values of all the variables in the dataset at each data point on our figure. To do so, we need to update each trace object as follows: 

In [15]:
# Define a hover-text generating function (returns a list of strings)
def make_text(X):
    return 'Country: %s\
    <br>Life Expectancy: %s years\
    <br>GDP per capita: %s $\
    <br>Population: %s million'\
    % (X['country'], X['lifeExp'], X['gdpPercap'], X['pop']/1e6)     
    
# Again, group data frame by continent sub-dataframe (named X),
#   make one trace object per continent and append to data object
i_trace = 0                                        # init. trace counter

for continent, X in df_year.groupby('continent'):
    text = X.apply(make_text, axis=1).tolist()     # get list of hover texts
    fig['data'][i_trace].update(text=text)         # update trace i
    i_trace += 1                                   # inc. trace counter

The `apply()` dataframe method loops through every row (with `axis=1`) and calls a given function, in our case `make_text()`.

Before sending the figure object to Plotly, add an annotation citing the data source.

In [16]:
# Update layout with an annotation object in 'annotations' (linked to a list)
fig['layout'].update(annotations=Annotations([
    Annotation(
        text='Data source: GapMinder 2007',  # annotation text
        showarrow=False,                     # remove arrow 
        xref='paper',   # use paper coords
        yref='paper',   #  for both x and y coordinates
        x=0.02,         # x-coord (slightly of plotting area edge)
        y=0.98,         # y-coord (slightly of plotting area edge)
        font=Font(size=14),   # increase font size (default is 12)
        bgcolor='#FFFFFF',    # white background
        borderpad=4           # set border/text space (in pixels)
    )
]))  

# Update title
title = "Fig 3.1b: Hans Rosling's Bubble Chart for the year {}".format(the_year)
fig['layout'].update(title=title)

c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.


c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:144: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


c:\users\carmt\appdata\local\programs\python\python37-32\lib\site-packages\plotly\graph_objs\_deprecations.py:66: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation

Layout({
    'annotations': [{'bgcolor': '#FFFFFF',
                     'borderpad': 4,
                     'font': {'size': 14},
                     'showarrow': False,
                     'text': 'Data source: GapMinder 2007',
                     'x': 0.02,
                     'xref': 'paper',
                     'y': 0.98,
                     'yref': 'paper'}],
    'autosize': False,
    'height': 500,
    'hovermode': 'closest',
    'plot_bgcolor': '#EFECEA',
    'showlegend': False,
    'title': {'text': "Fig 3.1b: Hans Rosling's Bubble Chart for the year 2007"},
    'width': 650,
    'xaxis': {'exponentformat': 'power',
              'gridcolor': '#FFFFFF',
              'showexponent': 'all',
              'ticklen': 8,
              'ticks': 'outside',
              'tickwidth': 1.5,
              'title': {'text': 'Gross Domestic Product per Capita [in USD of the year 2000]'},
              'type': 'log',
              'zeroline': False},
    'yaxis': {'gridcolor': '#F

This yields

In [17]:
# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='s3_life-gdp-log')    

Not a bad-looking plot. 

Spend some time hovering over the bubbles and admire Plotly's interactibility!

If you are feeling adventurous, jump to <a href="https://plot.ly/python/streaming-tutorial" target="_blank">section 7</a> to learn how to make an <a href="https://plot.ly/python/s7_streaming/s7_streaming_p3-bubbles.ipynb" target="_blank">annimated version</a> of Hans Rosling's bubble chart with Plotly's streaming API.

### 3.2 Small multiple bubble chart

Even though we managed to incorporate a lot of information in our first two bubble charts, only one year's worth of data could be displayed. Our next plot will feature the time evolution of the GDP per capita variable in the GapMinder dataset using a *small multiple*.

A small multiple is a series of similar graphics or charts plotted as subplots allowing them to be easily compared with one another. In our case, there will be one subplot per country. Each subplot will show GDP per capita versus time where the size of each data point will be a function of the life expectancy of that year. 

So first, find the start and end year as well as the number of countries in the dataset:

In [18]:
# Start year, end year and number of year is dataset
df['year'].min(), df['year'].max(), len(df['year'].unique())

(1952, 2007, 12)

In [19]:
countries = df['country'].unique()            # list of countries
N_country = len(countries)                    # number of countries
N_rowcol = int(np.ceil(np.sqrt(N_country)))   # size of the square subplot grid

N_country, N_rowcol  # print to screen

(142, 12)

There are 142 countries in the dataset and therefore, choosing to have a square small multiple for aesthetics, we need a 12 by 12 subplot grid to include every country. Consequently, there will be 2 empty subplots.

Next, generate a 12 by 12 subplot grid with shared x and y axes using `make_subplots()`:

In [20]:
# Generate Figure object with 144 axes (12 rows x 12 columns),
fig = tls.make_subplots(
    rows=N_rowcol,    # number of rows
    cols=N_rowcol,    # number of columns
    shared_xaxes=True,
    shared_yaxes=True
)

This is the format of your plot grid:
[ (1,1) x1,y1 ]      [ (1,2) x2,y1 ]      [ (1,3) x3,y1 ]      [ (1,4) x4,y1 ]      [ (1,5) x5,y1 ]      [ (1,6) x6,y1 ]      [ (1,7) x7,y1 ]      [ (1,8) x8,y1 ]      [ (1,9) x9,y1 ]      [ (1,10) x10,y1 ]    [ (1,11) x11,y1 ]    [ (1,12) x12,y1 ]  
[ (2,1) x1,y2 ]      [ (2,2) x2,y2 ]      [ (2,3) x3,y2 ]      [ (2,4) x4,y2 ]      [ (2,5) x5,y2 ]      [ (2,6) x6,y2 ]      [ (2,7) x7,y2 ]      [ (2,8) x8,y2 ]      [ (2,9) x9,y2 ]      [ (2,10) x10,y2 ]    [ (2,11) x11,y2 ]    [ (2,12) x12,y2 ]  
[ (3,1) x1,y3 ]      [ (3,2) x2,y3 ]      [ (3,3) x3,y3 ]      [ (3,4) x4,y3 ]      [ (3,5) x5,y3 ]      [ (3,6) x6,y3 ]      [ (3,7) x7,y3 ]      [ (3,8) x8,y3 ]      [ (3,9) x9,y3 ]      [ (3,10) x10,y3 ]    [ (3,11) x11,y3 ]    [ (3,12) x12,y3 ]  
[ (4,1) x1,y4 ]      [ (4,2) x2,y4 ]      [ (4,3) x3,y4 ]      [ (4,4) x4,y4 ]      [ (4,5) x5,y4 ]      [ (4,6) x6,y4 ]      [ (4,7) x7,y4 ]      [ (4,8) x8,y4 ]      [ (4,9) x9,y4 ]      [ (4,10) x10,y4 ]   

By default, `make_subplots()` generates subplots axes row-wise starting from the top-left corner of the plot. 

Next, define a trace-generating function. As before in [subsection 3.1](#3.1-A-Plotly-version-of-Hans-Rosling's-bubble-chart) the color of each bubble is determined by the continent and hover text is added using `make_text()`: 

In [42]:
# (!) Set bubble size monde et size reference
sizemode = 'diameter'
sizeref = df['lifeExp'].max() / 30

# Define a trace-generating function (returns a Scatter object)
def make_Scatter_gdp(i, j, x , y, color, sizes, name, text):
    return Scatter(
        x=x,   # x coordinates
        y=y,   # y coordinates
        name=name,        # label name (on hover)
        text=text,        # hover text
        mode='markers',   # marker pts only
        marker=Marker( 
            color=color,        # marker color
            size=sizes,         # maker sizes (a list)
            sizeref=sizeref,    # link sizeref
            sizemode=sizemode,  # link sizemode
            opacity=0.6         # partly transparent markers
        ),           
        xaxis= 'x{}'.format(i),  # (!) bind coordinate to given x-axis
        yaxis= 'y{}'.format(j)   # (!) bind coordinate to given y-axis
    )

To try to limit the amount of redundant information on the plot, only subplots on the left-hand side of the subplot grid will have labelled y axes and only subplots on the bottom row of the subplot grid will have labelled x axes.

Additionally, for better comparisons between countries, all subplots will have the same axis range. So, let's get an idea of the range in GDP per capita in the dataset:

In [43]:
df['gdpPercap'].min(), df['gdpPercap'].max()

(241.16587650000002, 113523.1329)

Then, define style dictionaries for the x and y axis objects and paste them into the figure object:

In [44]:
# For all y axes
yaxis_style = dict(
    type='log',                           # (!) log y-axis
    range=[np.log10(90), np.log10(5e5)],  # (!) set y-axis range w.r.t. log scale
    title='GDP per cap.', # title of the y axes
    ticks='outside',      # no ticks
    showline=True,        # show axis bounding line
    showgrid=False,       # remove grid
    zeroline=False        # no thick line at x=y=0
)

# For all x axes
xaxis_style = dict(
    range=[1950, 2010],  # set x-axis range
    title='year',        # title of the x axes
    ticks='outside',     # no ticks
    showline=True,       # show axis bounding line
    showgrid=False,      # remove grid
    zeroline=False       # no thick line at x=y=0
)

# Paste the style dictionaries into the figure object
for i in range(N_rowcol):
    yaxis_splt = fig['layout']['yaxis{}'.format(i+1)]
    yaxis_splt.update(yaxis_style)
    xaxis_splt = fig['layout']['xaxis{}'.format(i+1)]
    xaxis_splt.update(xaxis_style)

Now, make an annotation-generating function to label the subplots with each country's name:

In [91]:
# Define an annotation-generating function, to label each subplot
def make_splt_anno(i, j, country):
    rv=[]
    if len(country) > 14:
        country = country[0:14] + '.'  # truncate country's name if too long
    
    xref='x{}'.format(i),   # position in relation to the x
    yref='y{}'.format(j),   #   and y axes
    x=1955,                 # x position
    xanchor='left',         #   and anchor
    y=np.log10(2.5e5),      # y position (in log coords)
    text=country,           # text is the country name
    showarrow=False,        # no arrow 
    align='center',         # align text in the center
    font=Font(size=14),     # font size
    
    rv = ['xref',xref], ['yref',yref], ['x',x], ['xanchor',xanchor], ['y',y], ['text',text], ['showarrow',showarrow], ['align',align], ['font',font]
    print(rv)
    rv = list(zip(one, two, three, four, five, six, seven, eight, nine))
    return (rv)

Add a few keys to the layout object:

In [57]:
width = 2000   # fix plot's width 
height = 1800  #  and height in pixels

title = "Fig 3.2: GDP per Capita from 1952 to 2007 \
in USD of the year 2000 [GapMinder]"                  # plot's title

fig['layout'].update(
    title=title,              # set plot's title
    titlefont=Font(size=30),  # increase title font size
    showlegend=False,         # remove legend
    autosize=False,           # turn off autosize
    width=width,              # plot's width
    height=height             # plot's height
)
    
fig['layout']['annotations'] = Annotations([])  # init. 'annotations' key

Now, loop through every country in the dataframe and fill in the data object and update each axis' style:

In [47]:
fig

Figure({
    'data': [],
    'layout': {'autosize': False,
               'height': 1800,
               'showlegend': False,
               'title': {'text': 'Fig 3.2: GDP per Capita from 1952 to 2007 in USD of the year 2000 [GapMinder]'},
               'width': 2000,
               'xaxis': {'anchor': 'y12',
                         'domain': [0.0, 0.06805555555555555],
                         'range': [1950, 2010],
                         'showgrid': False,
                         'showline': True,
                         'ticks': 'outside',
                         'title': {'text': 'year'},
                         'zeroline': False},
               'xaxis10': {'anchor': 'free',
                           'domain': [0.7625, 0.8305555555555555],
                           'position': 0.0,
                           'range': [1950, 2010],
                           'showgrid': False,
                           'showline': True,
                           'ticks': 'outside',
   

In [97]:
i = j = 1  # init. col and row counters

# Group dataframe by country in alphabetical order and loop
for country, X in df.groupby('country'):
    
    x = X['year'].values                        # get years
    y = X['gdpPercap'].values                   # get GDP values
    sizes = X['lifeExp'].values                 # get bubble size
    continent = X['continent'].values[0]        # get continent name
    color = colors[continent]                   # get bubble color
    text = X.apply(make_text, axis=1).tolist()  # get hover text

    # Append data object
    #fig['data'].append(make_Scatter_gdp(i, j, x, y, 
    #                                    color, sizes, 
    #                                   continent, text))
    
   # try:
    trace1 = make_Scatter_gdp(i, j, x, y, color, sizes, continent, text) 
    fig.append_trace(trace1, i, j)
    
        # (!) fig['data'] was intialized by tls.make_subplots()
    
    
    if len(country) > 14:
        xcountry = country[0:14] + '.'  # truncate country's name if too long
    
    # Append annotations object, label each subplot
    #fig['layout'].update(list(annotations=dict(
    #    xref='x{}'.format(i),   # position in relation to the x
    #    yref='y{}'.format(j),   #   and y axes
    #    x=1955,                 # x position
    #    xanchor='left',         #   and anchor
    #    y=np.log10(2.5e5),      # y position (in log coords)
    #    text=country,           # text is the country name
    #    showarrow=False,        # no arrow 
    #    align='center',         # align text in the center
    #    font=14
    #    )))              # font size
    
        # Increment counter accordingly
    if i == 12:
        j += 1
        i = 1
    else:
        i += 1
   # except:
   #     print('exited')
   #     break

We are now ready to send the figure object to Plotly and see the results:

In [86]:
# (@) Send figure object to Plotly, show plot in notebook
py.iplot(fig, filename='s3_small-multiples', width=width, height=height)       

# adjust output cell with 'width' and 'height'

To see this figure in separate tab, use this <a href="https://plot.ly/~etpinard/310.embed" target="_blank">link</a>.


Who wants to print this on a T-shirt?

<hr>
    
<h4 style="font-size:14pt;">Go to

[Section 4 --- Histograms and Box Plots](https://plot.ly/python/histograms-and-box-plots-tutorial) </h4>

<h4 style="font-size:14pt;">Go to

[Section 2 --- Bar Charts](https://plot.ly/python/bar-charts-tutorial) </h4>

<h4 style="font-size:14pt;">Go to

[top of page](#Plotly's-Python-API-User-Guide) </h4>

<h4 style="font-size:14pt;">Go to User Guide's

[homepage](https://plot.ly/python/user-guide) </h4>
    
<hr>

<div style="float:right; \">
    <img src="http://i.imgur.com/4vwuxdJ.png" 
 align=right style="float:right; margin-left: 5px; margin-top: -10px" />
</div>

<h4>Got Questions or Feedback? </h4>

About <a href="https://plot.ly" target="_blank">Plotly</a>

* email: feedback@plot.ly 
* tweet: 
<a href="https://twitter.com/plotlygraphs" target="_blank">@plotlygraphs</a>

About the <a href="https://github.com/plotly/python-user-guide" target="_blank">User Guide</a>
 
* email: etienne@plot.ly
* tweet: <a href="https://twitter.com/etpinard" target="_blank">@etpinard</a>

<h4 style="margin-top:30px;">Notebook styling ideas</h4>

Big thanks to

* <a href="http://nbviewer.ipython.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Prologue/Prologue.ipynb" target="_blank">Cam Davidson-Pilon</a>
* <a href="http://lorenabarba.com/blog/announcing-aeropython/#.U1ULXdX1LJ4.google_plusone_share" target="_blank">Lorena A. Barba</a>

<br>

In [27]:
# CSS styling within IPython notebook
from IPython.display import display, HTML
display(HTML(open('../custom.css').read()))